# XGBoost

In [30]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import pygad
from sklearn.model_selection import cross_val_score

In [20]:
df = pd.read_csv("winequality-red.csv")
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [21]:
features_train = train_data.copy()
labels_train = features_train.pop('quality')
features_train = np.array(features_train)
print("Train set : {train_set} , shape: {size}".format(train_set=features_train,size=features_train.shape))


features_test = test_data.copy()
labels_test = features_test.pop('quality')
features_test = np.array(features_test)
print("Test set : {test_set} , shape: {size}".format(test_set=features_test,size=features_test.shape))

Train set : [[ 8.7   0.69  0.31 ...  3.48  0.74 11.6 ]
 [ 6.1   0.21  0.4  ...  3.25  0.59 11.9 ]
 [10.9   0.39  0.47 ...  3.3   0.75  9.8 ]
 ...
 [ 7.2   0.62  0.06 ...  3.51  0.54  9.5 ]
 [ 7.9   0.2   0.35 ...  3.32  0.8  11.9 ]
 [ 5.8   0.29  0.26 ...  3.39  0.54 13.5 ]] , shape: (1279, 11)
Test set : [[ 7.7    0.56   0.08  ...  3.24   0.66   9.6  ]
 [ 7.8    0.5    0.17  ...  3.39   0.48   9.5  ]
 [10.7    0.67   0.22  ...  3.28   0.98   9.9  ]
 ...
 [ 8.3    0.6    0.25  ...  3.15   0.53   9.8  ]
 [ 8.8    0.27   0.39  ...  3.15   0.69  11.2  ]
 [ 9.1    0.765  0.04  ...  3.29   0.54   9.7  ]] , shape: (320, 11)


In [28]:
xg_reg= XGBRegressor()
xg_reg.fit(features_train, labels_train)


In [37]:
y_pred= xg_reg.predict(features_test)
mse= mean_squared_error(labels_test,y_pred)

print("MSE: %0.2f" % (mse))
rmse= np.sqrt(mse)
print("RMSE: %0.2f" % (rmse))

errors = abs(y_pred - labels_test)# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / labels_test)# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

MSE: 0.35
RMSE: 0.59
Mean Absolute Error: 0.42 degrees.
Accuracy: 92.41 %.


## Cross validation

In [39]:
X = df.copy()
y= X.pop('quality')
X = np.array(X)


model = XGBRegressor(objective="reg:squarederror")
scores= cross_val_score(model,X,y,scoring="neg_mean_squared_error",cv=10)
rmse=np.sqrt(-scores)

print(rmse)


[0.63417291 0.64731861 0.7388265  0.71471989 0.64101149 0.79886101
 0.71919869 0.73148063 0.69219977 0.69565355]


In [26]:

desidered_output=10
def fitness_func(ga_instance,solution,solution_idx):
    input_array=np.array(solution)
    input_array_for_prediction = np.expand_dims(input_array,axis=0)
    output= xg_reg.predict(input_array_for_prediction)
    fitness = 1.0 / np.abs(output - desidered_output)
    return fitness


num_generations =50
num_parents_mating = 4

sol_per_pop = 8
num_genes = 11

init_range_low = -1
init_range_high = 10

gene_space = [np.arange(4.0, 15.0, 0.1),np.arange(0, 0.9, 0.01),np.arange(0, 1, 0.01),
              np.arange(0, 18.0, 0.1),np.arange(0.05, 0.5, 0.001),
              np.arange(2, 60, 1),np.arange(6, 160, 1),np.arange(0.800, 1.100, 0.001),np.arange(2.8, 4.0, 0.1),
              np.arange(0.10, 2.00, 0.01),np.arange(8.0, 15.0, 0.1)]

parent_selection_type = "sss"
keep_parents = 1

crossover_type = "single_point"

mutation_type = "random"
mutation_percent_genes = 10

ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_func,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       gene_space=gene_space,
                       #init_range_low=init_range_low,
                       #init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes)

ga_instance.run()



/home/niccolo/python-projects/optimization/env/lib/python3.10/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


In [27]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Parameters of the best solution : {solution}".format(solution=solution))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))

input_array=np.array(solution)
input_array_for_prediction = np.expand_dims(input_array,axis=0)
    
prediction =  xg_reg.predict(input_array_for_prediction)
print("Predicted output based on the best solution : {prediction}".format(prediction=prediction))

Parameters of the best solution : [ 7.8    0.3    0.17  10.     0.073  7.     9.     0.962  3.8    1.73
 13.3  ]
Fitness value of the best solution = [0.77295494]
Predicted output based on the best solution : [8.706264]
